# Default Import

In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2021-06-25 15:42 | by soge | Using ris library version: 2.2.1 | File Location: E:\RIS\Staff Folders\Samuel\Requests\SH\Speed Limits\0521_Speed_Limits_Update

# Db connections

In [2]:
try: 
    db.params['user']
except:
    db = pysqldb.DbConnect(type='PG', server='dotdevrhpgsql01', database='CRASHDATA')


Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Attempted to remove 1 expired temp tables: [('public', 'archive_speed_limit_20d_61621')]


In [3]:
#sip = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG', user = db.params['user'], password = db.params['password'])
#forms = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

#BACK UP SPEED LIMITS FILE
#CREATE A TABLE WITH SPEED LIMITS DATA AND TODAY'S DATE
#i.e Archive Copy

In [4]:
db.query("""
CREATE TABLE archive_speed_limit_20d_62521 AS

SELECT * FROM speed_limit_20d;

GRANT ALL ON archive_speed_limit_20d_62521 TO PUBLIC;
""")

- Query run 2021-06-25 15:43:08.762000
 Query time: Query run in 155000 microseconds 
 * Returned 0 rows *


In [5]:
#Streets: 

all_segs = pd.read_excel('sl_limits_update_all.xlsx')

In [6]:
all_segs.street.unique()

array([u'ASTORIA BOULEVARD', u'ASTORIA BOULEVARD NORTH',
       u'ASTORIA BOULEVARD SOUTH', u'CONDUIT BOULEVARD',
       u'CROSS BAY BOULEVARD', u'HYLAN BOULEVARD',
       u'HYLAN BOULEVARD NORTH', u'HYLAN BOULEVARD SOUTH',
       u'JEROME AVENUE', u'NORTH CONDUIT AVENUE', u'PELHAM PARKWAY',
       u'SOUTH CONDUIT AVENUE', u'THROGS NECK EXPRESSWAY SR',
       u'VAN WYCK EXPRESSWAY', u'VAN WYCK EXPRESSWAY SR EAST',
       u'VAN WYCK EXPRESSWAY SR WEST', u'VAN WYCK EXPWY NB EN 138 ST',
       u'WOODHAVEN BOULEVARD'], dtype=object)

In [7]:
astoria = tuple(map(int,list(all_segs.loc[all_segs.street.str.contains("ASTORIA")].segmentid)))
ns_conduit = tuple(map(int,list(all_segs.loc[(all_segs.street.str.contains("NORTH CONDUIT")) | (all_segs.street.str.contains("SOUTH CONDUIT"))].segmentid)))
hylan = tuple(map(int,list(all_segs.loc[all_segs.street.str.contains("HYLAN")].segmentid)))
jerome = tuple(map(int,list(all_segs.loc[all_segs.street.str.contains("JEROME")].segmentid)))
pelham = tuple(map(int,list(all_segs.loc[all_segs.street.str.contains("PELHAM")].segmentid)))
throgs = tuple(map(int,list(all_segs.loc[all_segs.street.str.contains("THROGS")].segmentid)))
van_wyck = tuple(map(int,list(all_segs.loc[all_segs.street.str.contains("VAN WYCK")].segmentid)))
woodhaven = tuple(map(int,list(all_segs.loc[all_segs.street.str.contains("WOODHAVEN")].segmentid)))
conduit = tuple(map(int,list(all_segs.loc[(all_segs.street.str.contains("CONDUIT")) & (all_segs.street.str.contains("NORTH") == False)  & (all_segs.street.str.contains("SOUTH") == False)].segmentid)))
crossbay = tuple(map(int,list(all_segs.loc[all_segs.street.str.contains("CROSS BAY")].segmentid)))


In [21]:
print tuple(ns_conduit)

(50985, 51129, 51138, 51145, 51148, 51149, 51300, 51417, 51423, 51428, 51552, 51554, 51561, 51902, 51905, 52066, 52078, 54608, 54610, 54688, 54746, 54749, 54752, 54844, 54845, 54853, 54953, 54956, 54960, 54962, 55075, 55076, 55099, 55102, 55251, 55256, 55257, 55260, 55373, 55538, 55543, 55546, 58625, 58628, 58638, 58777, 58780, 58781, 58784, 58785, 58916, 58919, 59033, 59044, 59051, 59144, 59148, 59152, 59285, 59293, 59302, 59447, 59457, 59458, 59569, 59582, 59854, 62764, 62938, 63066, 63067, 63256, 103499, 105002, 106666, 106667, 107762, 110564, 111697, 114604, 115649, 116181, 116182, 118876, 118877, 118896, 118897, 119035, 119036, 134848, 134877, 134878, 136088, 169096, 169097, 169199, 170014, 170015, 170116, 170117, 170118, 170121, 170798, 170799, 171369, 171370, 171371, 171402, 171403, 183732, 183733, 183734, 183735, 184980, 184981, 187140, 188564, 188565, 237627, 257129, 257130, 257131, 257132, 258648, 258649, 273832, 273833, 273834, 277375, 277376, 286627, 286628, 9009712, 900971

In [13]:
mph_30 = tuple(ns_conduit + hylan)
mph_25 = tuple(pelham + throgs + jerome + woodhaven + astoria + conduit + hylan + van_wyck)

### Inserting into Speed limits table if necessary

insert = db.dfquery("""
INSERT INTO speed_limit_19d(segmentid, street, postvz_sl, postvz_sg, prevz_sl, prevz_sg, version, geom)
SELECT segmentid::int , street, 0 postvz_sl, 'NO'::varchar postvz_sg, 0 prevz_sl, 'NO'::varchar _prevz_sg, '19d' "version", geom
FROM LION
WHERE segmentid ::int IN  ('0312733', '0312802', '0312769')
""")

In [14]:
x = db.dfquery("""
SELECT segmentid, street, postvz_sl, postvz_sg, prevz_sl, prevz_sg, 
       version, geom
FROM public.speed_limit_20d
WHERE segmentid ::int IN  {} p
""".format(mph_30))

- Query failed: syntax error at or near "p"
LINE 5: ...3, 194153, 194156, 194365, 194366, 262049, 262050, 170491) p
                                                                      ^

	
- Query run 2021-06-03 18:25:41.465000
	
SELECT segmentid, street, postvz_sl, postvz_sg, prevz_sl, prevz_sg, 
       version, geom
FROM public.speed_limit_20d
WHERE segmentid ::int IN  (134624, 134638, 139113, 178460, 178461, 178504, 178565, 178566, 178567, 193966, 193969, 193976, 193978, 193979, 193981, 194042, 194046, 194052, 194053, 194054, 194257, 196622, 196626, 231426, 231427, 238955, 238956, 253892, 253893, 253897, 253898, 262073, 262074, 262076, 262077, 9008825, 9008826, 9008829, 9008830, 93627, 93641, 93643, 93677, 93679, 93746, 93749, 93754, 93757, 93762, 93769, 93771, 93772, 93783, 93785, 93906, 93909, 93912, 94780, 116952, 116953, 131828, 179537, 179538, 179545, 179546, 187039, 187040, 108978, 108977, 9008872, 9008871, 108288, 135200, 135201, 144606, 144607, 146536, 146537, 146538, 1465

In [20]:
y = db.query("""UPDATE public.speed_limit_20d
                SET postvz_sl = 30
                WHERE segmentid in {}
                """.format(tuple(hylan)))

- Query run 2021-06-25 15:58:44.492000
 Query time: Query run in 317000 microseconds 
 * Returned 0 rows *


In [ ]:
"""

"street" like 'PELHAM PARKWAY'
or "street" like 'WHITE PLAINS ROAD’
or “street" like ‘STILLWELL AVENUE’

or

"street" like 'THROGS NECK EXPRESSWAY'
or "street" like 'LONGSTREET AVENUE'
or "street" like 'SAMPSON AVENUE'

or

"street" like 'JEROME AVENUE'
or "street" like 'BAINBRIDGE AVENUE'
or "street" like 'EAST 233 STREET'

or

"street" like 'WOODHAVEN BOULEVARD'
or "street" like 'QUEENS BOULEVARD'
or "street" like 'ROCKAWAY BOULEVARD'

or

—NORTH SERVICE ROAD??
"street" like 'NORTH SERVICE ROAD'
or "street" like 'ROCKAWAY BOULEVARD'
or "street" like 'NORTH CHANNEL BRIDGE'

or

"street" like 'ASTORIA BOULEVARD'
or "street" like '111 STREET'
or "street" like '8 STREET'

or

"street" like 'SOUTH CONDUIT AVENUE'
or "street" like 'ATLANTIC AVENUE'
or "street" like 'SUNRISE HIGHWAY'

or

"street" like 'HYLAN BOULEVARD'
or "street" like 'BAY STREET'
or "street" like 'MASSACHUSETTS STREET'

or

"street" like '%VAN WYCK%'
or "street" like '135 AVENUE'
or "street" like 'QUEENS BOULEVARD'

"""